In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def conv_layer(input, channels_in, channels_out, name='conv'):
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal(
            [5, 5, channels_in, channels_out], stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[channels_out]), name='B')
        conv = tf.nn.conv2d(input, filter=weights, strides=[
                            1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv + biases)
        tf.summary.histogram('weights', weights)
        tf.summary.histogram('biases', biases)
        tf.summary.histogram('activations', act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 简化全连接层


def fc_layer(input, num_inputs, num_outputs, name='fc'):
    with tf.name_scope(name):
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs],
                                                  stddev=0.05), name='W')
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]), name='B')
        act = tf.matmul(input, weights) + biases

        tf.summary.histogram('weights', weights)
        tf.summary.histogram('biases', biases)
        tf.summary.histogram('activations', act)

        return act

保存到新的目录

In [5]:
import os

In [6]:
tensorboard_dir = 'tensorboard/mnist4/'   # 保存目录
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)

根据不同参数构建模型

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST', one_hot=True)
def mnist_model(learning_rate, use_two_fc, use_two_conv, hparam):
    tf.reset_default_graph()    # 重置计算图
    sess = tf.Session()
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    if use_two_conv:    # 是否使用两个卷积
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        # 如果使用一个卷积，则再添加一个max_pooling保证维度相通
        conv1 = conv_layer(x_image, 1, 64, "conv")
        conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[
                                  1, 2, 2, 1], padding="SAME")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])

    if use_two_fc:    # 是否使用两个全连接
        fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
        relu = tf.nn.relu(fc1)
        tf.summary.histogram("fc1/relu", relu)
        logits = fc_layer(fc1, 1024, 10, "fc2")
    else:
        logits = fc_layer(flattened, 7*7*64, 10, "fc")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()    # 收集所有的summary

    saver = tf.train.Saver()     # 保存训练过程

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(tensorboard_dir + hparam)
    writer.add_graph(sess.graph)

    for i in range(2001):
        batch = data.train.next_batch(100)
        if i % 5 == 0:   # 每5轮写入一次
            # 同上，feed_dict应该使用验证集，但是当前仅作为演示目的，在此不做修改
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={
                                           x: batch[0], y: batch[1]})
            writer.add_summary(s, i)

        if i % 100 == 0:    # 每100轮保存依存训练过程
            train_accuracy = sess.run(
                accuracy, feed_dict={x: batch[0], y: batch[1]})
            saver.save(sess, os.path.join(tensorboard_dir, "model.ckpt"), i)

            print("迭代轮次: {0:>6}, 训练准确率: {1:>6.4%}".format(i, train_accuracy))

        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
# 用于生成超参数的字符串
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

开始训练

In [9]:
for learning_rate in [1E-3, 1E-4, 1e-5]:
    for use_two_fc in [False, True]:
        for use_two_conv in [False, True]:
            hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
            print('Starting run for %s' % hparam)

            mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)

print('Done training!')

Starting run for lr_1E-03,conv=1,fc=1
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

迭代轮次:      0, 训练准确率: 11.0000%
迭代轮次:    100, 训练准确率: 87.0000%
迭代轮次:    200, 训练准确率: 94.0000%
迭代轮次:    300, 训练准确率: 95.0000%
迭代轮次:    400, 训练准确率: 96.0000%
迭代轮次:    500, 训练准确率: 94.0000%
迭代轮次:    600, 训练准确率: 98.0000%
迭代轮次:    700, 训练准确率: 98.0000%
迭代轮次:    800, 训练准确率: 96.0000%
迭代轮次:    900, 训练准确率: 99.0000%
迭代轮次:   1000, 训练准确率: 97.0000%
迭代轮次:   1100, 训练准确率: 99.0000%
迭代轮次:   1200, 训练准确率: 99.0000%
迭代轮次:   1300, 训练准确率: 96.0000%
迭代轮次:   1400, 训练准确率: 98.0000%
迭代轮次:   1500, 训练准确率: 97.0000%
迭代轮次:   1600, 训练准确率: 97.0000%
迭代轮次:   1700, 训练准确率: 95.0000%
迭代轮次:   1800, 训练准确率: 98.0000%
迭代轮次:   1900, 训练准确率: 98.0000%
迭代轮次:   2000, 训练准确率: 99.0000%
Starting run for lr_1E-03,conv=2,fc=1
迭代轮次:      0, 训练准确率: 11.0000%
迭代轮次:    100, 训练准确率: 96.0000%
迭代轮次:    200, 训练准确率: 97.0000%
迭代轮次:    300,

迭代轮次:   1500, 训练准确率: 87.0000%
迭代轮次:   1600, 训练准确率: 98.0000%
迭代轮次:   1700, 训练准确率: 96.0000%
迭代轮次:   1800, 训练准确率: 90.0000%
迭代轮次:   1900, 训练准确率: 95.0000%
迭代轮次:   2000, 训练准确率: 90.0000%
Done training!
